# Python Script

## Template 1 - Running Hyper Parameter Grid Search

In [131]:
def python_script(resample, fold, desc, prop):
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st"
    python_script=f"{desc}_{prop}_{resample}_{fold}_st.py"
    with open(f'{dir}/{python_script}','w') as f:
        f.write("""# Part one General and Data Hanlding

import numpy as np
import pandas as pd
import os
from pathlib import Path

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

# Part 2 Model

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import layers, models, initializers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger



# Data Block Functions

def path_fold(home,resample,i_fold):
    path="{}/CV/{}/fold_{}".format(os.getcwd(),resample,i_fold)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}/{i_fold}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def path_resample(home,resample):
    path="{}/CV/{}/".format(os.getcwd(),resample)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def cv_hp(df,home):
    resample_split  = ShuffleSplit(50, test_size=0.3, random_state=1)
    fold_split      = ShuffleSplit(5 , test_size=0.3, random_state=1)
    train_val_split = ShuffleSplit(1 , test_size=0.3, random_state=1)
    
    for resample, (train_val_index, test_index) in enumerate(resample_split.split(df)):
        train_val = pd.DataFrame(df['ID'].iloc[train_val_index])
        test = pd.DataFrame(df['ID'].iloc[test_index])
        for i, (train_index, val_index) in enumerate(train_val_split.split(train_val)):
            train = pd.DataFrame(df['ID'].iloc[train_index])
            val   = pd.DataFrame(df['ID'].iloc[val_index])
        resample_path = path_resample(home,resample)
        train.to_csv(f'{resample_path}/train.csv')
        val.to_csv(f'{resample_path}/val.csv')
        test.to_csv(f'{resample_path}/test.csv')
        # train,val,test to_csv
        for i_fold, (train_val_fold_index, test_fold_index) in enumerate(fold_split.split(train)):
            train_val_fold = pd.DataFrame(train['ID'].iloc[train_val_fold_index])
            test_fold = pd.DataFrame(train['ID'].iloc[test_fold_index])
            for i, (train_fold_index, val_fold_index) in enumerate(train_val_split.split(train_val_fold)):
                train_fold = pd.DataFrame(train_val_fold['ID'].iloc[train_fold_index])
                val_fold   = pd.DataFrame(train_val_fold['ID'].iloc[val_fold_index])
            i_fold_path = path_fold(home,resample,i_fold)
            train_fold.to_csv(f'{i_fold_path}/train.csv')
            val_fold.to_csv(f'{i_fold_path}/val.csv')
            test_fold.to_csv(f'{i_fold_path}/test.csv')
            

    return print("data organised into 50 CV with 5-fold inner CV")


# Accessing Data Via ID CV and Full table functions

def access_fold_csv(df,home,resample,fold):
    df_path = path_fold(home,resample,fold)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


def access_resample_csv(df,home,resample):
    df_path = path_resample(home,resample)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


# Loading X and Y Data Functions

def padding(X_descr_train_scaled):
#     Padding function so X data is always 250 dimensions
# Must be coupled with load_data. NB! double check if the scalling is not affected
# https://www.geeksforgeeks.org/python-call-function-from-another-function/
    a=X_descr_train_scaled.to_numpy()
    b=np.zeros((len(X_descr_train_scaled), 
                (250-int(X_descr_train_scaled.to_numpy().shape[1]))
               )
              )
    padded=np.concatenate((a,b),
                           axis=1, 
                          out=None, 
                          dtype=None
                         )
    return padded


def load_xy(file,desc):
    # Universal funciton for loading
# y_1, y_2, y_3, y_4 and x data from input csv (All, Train, Val or Train)
    y_1 = file[['dH']].copy()
    y_2 = file[['dS']].copy()
    y_3 = file[['dG']].copy()
    y_4 = file[['Tm']].copy()

    Y = file[['dH','dS','dG','Tm']].copy()
    # Convert y data into required input shape
    y_1 = y_1.to_numpy()
    y_1 = y_1.reshape(y_1.shape[0])
    y_2 = y_2.to_numpy()
    y_2 = y_2.reshape(y_2.shape[0])
    y_3 = y_3.to_numpy()
    y_3 = y_3.reshape(y_3.shape[0])
    y_4 = y_4.to_numpy()
    y_4 = y_4.reshape(y_4.shape[0])
    
    # Load features based on prop
    X = file[[col for col in file.columns if f'{desc}_'in col]]
    
    return y_1, y_2, y_3, y_4, Y, padding(X), X



# Model and Scoring Functions

def r2_func(y_true, y_pred, **kwargs):
    return metrics.r2_score(y_true, y_pred)
def rmse_func(y_true, y_pred, **kwargs):
    return np.sqrt(metrics.mean_squared_error(y_true, y_pred))  
def bias_func(y_true, y_pred, **kwargs):
    return np.mean(y_true-y_pred)
def sdep_func(y_true, y_pred, **kwargs):
    return (np.mean((y_true-y_pred-(np.mean(y_true-y_pred)))**2))**0.5
#these 4 are for tensorflow formats
def r2_func_tf(y_true, y_pred, **kwargs):
    numerator = tf.reduce_sum(tf.square(y_true - y_pred))
    denominator = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    r2 = 1 - numerator / denominator
    return r2
def rmse_func_tf(y_true, y_pred, **kwargs):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    rmse = tf.sqrt(mse)
    return rmse
def bias_func_tf(y_true, y_pred, **kwargs):
    bias = tf.reduce_mean(y_true - y_pred)
    return bias
def sdep_func_tf(y_true, y_pred, **kwargs):
    diff = y_true - y_pred
    mean_diff = tf.reduce_mean(diff)
    sdep = tf.sqrt(tf.reduce_mean(tf.square(diff - mean_diff)))
    return sdep

# Model


def build_model(hp):
    
#     Hyper parameters         
    model_type1 = hp.Choice("model_type1", ["CNN3","CNN2","CNN1"])
    model_type = hp.Choice("model_type", ["Dense3"])
 
    
#     INPUT for NN
    
    inputs = keras.Input(shape=(250,1))
    x_layer=inputs
    
#     MANDATORY CNN (optional to move into first condition hp.cond_scope
    # with hp.conditional_scope("model_type1", ["CNN0"]):
    #         if model_type1 == "CNN0":
    #             pass
#     CONDITIONAL CONVOLUTION LAYERS (Consider moving the above into CNN1) test 0-3 CNN and 0-3 Dense
    with hp.conditional_scope("model_type1", ["CNN1","CNN2""CNN3"]):
            if model_type1 != "CNN0":
                x_layer = keras.layers.Conv1D(32, 
                        kernel_size=(3), 
                        strides=(2), 
                        padding='valid', 
                        activation='relu', 
                        input_shape=(250,1),
                        name = 'conv1d_1'
                        )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = 'maxpooling_1')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = 'batchnorm_1')(x_layer)
                pass
                
            if model_type1 != "CNN1":
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_2'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_2')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_2')(x_layer)

            if model_type1 != "CNN1" or "CNN2":               
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_3'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_3')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_3')(x_layer)
                
#     FLATTEN AFTER CONVOLUTIONS
    x_layer = keras.layers.Flatten(name = 'flatten')(x_layer)
    
#     CONDITIONAL DENSE LAYERS
    # with hp.conditional_scope("model_type", ["Dense0"]):
    #     if model_type == "Dense0":
    #         pass
            
    with hp.conditional_scope("model_type", ["Dense3"]): #["Dense1","Dense2","Dense3"]
        if model_type != "Dense0":
            hp_layer_1= hp.Choice(f'layer_1', values = [16])#values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_1,
                        activation='relu',
                        use_bias=True,
                        # name='layer_1',
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
        if model_type != "Dense1":
            hp_layer_2_2= hp.Choice(f'layer_2_2', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_2_2,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)

        if model_type != "Dense1" or "Dense2":
            hp_layer_3_3= hp.Choice(f'layer_3_3',  values = [16]) #values=[16,32,64])
            
            x_layer = keras.layers.Dense(
                        hp_layer_3_3,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
#     OUTPUT LAYERS

    # output_1 = keras.layers.Dense(1, name='enthalpy_pred')(x_layer)
    # output_2 = keras.layers.Dense(1, name='entropy_pred')(x_layer)
    # output_3 = keras.layers.Dense(1, name='free_energy_pred')(x_layer)
    # output_4 = keras.layers.Dense(1, name='melting_temperature')(x_layer)

    # output_1 = keras.layers.Dense(1, name='dH')(x_layer)
    # output_2 = keras.layers.Dense(1, name='dS')(x_layer)
    # output_3 = keras.layers.Dense(1, name='dG')(x_layer)
    # output_4 = keras.layers.Dense(1, name='Tm')(x_layer)
    

    # model = Model(inputs=inputs, outputs=[output_1, output_2, output_3, output_4])
    output_1 = keras.layers.Dense(1, name='output')(x_layer)
    model = Model(inputs=inputs, outputs=output_1)
    
#     SETTINGS
#     SETTINGS

#     ADAPTIVE LEARNING RATE   
    
    initial_learning_rate = 0.01
    decay_steps = 10.0
    decay_rate = 0.5
    learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(
                                    initial_learning_rate, decay_steps, decay_rate)
    
#     SETTING ADAM OPTIMISER
    optimiser = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
    
#     COMPILE MODEl
    model.compile(loss = "mse" , 
                  optimizer = optimiser, 
                  metrics = ["mse",'mean_absolute_error',r2_func_tf, rmse_func_tf, bias_func_tf, sdep_func_tf])   
    
    return model
""")
        
        f.write(f"""              

# Code Execution 

df=pd.read_csv("Lomzov_dataset_IY.csv")
home=os.getcwd()
# data Generation
# cv_hp(df,home)

# Actual instrucitons
resample={resample}
fold={fold}
GSHT_list=['dH','dS','dG','Tm']
desc='{desc}'
prop = '{prop}' 
model_name = f"1DConv_st_{prop}" 
epochs = 300
batch  = 16

""")

        f.write("""
# train, val, test = access_resample_csv(df,home,resample)
train_fold, val_fold, test_fold = access_fold_csv(df,home,resample,fold)

y_1_train, y_2_train, y_3_train, y_4_train, Y_train, X_padded_train, X_train = load_xy(train_fold,desc)
y_1_val,   y_2_val,   y_3_val,   y_4_val,   Y_val,   X_padded_val,   X_val   = load_xy(val_fold,desc)
y_1_test,  y_2_test,  y_3_test,  y_4_test,  Y_test,  X_padded_test,  X_test  = load_xy(test_fold,desc)

fold_path = path_fold(home,resample,fold)
# resample_path = path_resample(home,resample)

# Define the directory path
directory_path = Path(f"{fold_path}/{desc}/{model_name}/")

tunner_path      = Path(f'{directory_path}/tunner')
csv_logger_path  = Path(f'{directory_path}/csv_logger/')
cp_callback_path = Path(f'{directory_path}/model_checkpoint/')
tensorboard_path = Path(f'{directory_path}/tensorboard_logs/')

# Ensure the directory exists, create it if necessary
tunner_path.mkdir(parents=True, exist_ok=True)
csv_logger_path.mkdir(parents=True, exist_ok=True)
cp_callback_path.mkdir(parents=True, exist_ok=True)
tensorboard_path.mkdir(parents=True, exist_ok=True)


tuner = kt.GridSearch(build_model,
                   objective=kt.Objective('val_loss', 'min'),
                    # loss = 'val_loss',
                   # objective = ['val_mse','val_free_energy_pred_mse'],
                  directory=tunner_path,
                  overwrite=False,
                  project_name=f'{batch}')

with open(f'{tunner_path}/tuner_path.txt', 'w') as f:
    f.write(tuner.project_dir)
f.close

#### CALL BACKS!
es = EarlyStopping(monitor      = 'val_loss', 
                        mode     = 'min', 
                        verbose  = 1, 
                        patience = 2000, 
                    restore_best_weights = True)
# CSV Logger
csv_logger = CSVLogger(f'{csv_logger_path}/model_history.csv' , append=True)

# CP_callbacks      not required when using a tunner       
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'{cp_callback_path}/cp.ckpt',
#                                                  save_weights_only=True,
#                                                  verbose=1)

# TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_path, 
                                                       #/{batch}', # _ADAPTIVELEARNIGNRATE_01_10_Dense3_64_3CNN_lr_3_es
                                                      update_freq = 1,
                                                      # histogram_freq=1, 
                                                      write_graph=False, 
                                                      write_images=False)
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard

# Covert to list and provide to Keras Regressor
keras_callbacks = [es, csv_logger, tensorboard_callback]

# Execution

ys_train = [y_1_train,y_2_train,y_3_train,y_4_train] 
ys_val   = [y_1_val,y_2_val,y_3_val,y_4_val]
ys_test  = [y_1_test,y_2_test,y_3_test,y_4_test]
GSHT_list.index(f'{prop}')

history=tuner.search(X_padded_train, ys_train[GSHT_list.index(f'{prop}')],
            epochs = epochs,
            batch_size=batch,
            verbose = 3,
            validation_data =(X_padded_val, ys_val[GSHT_list.index(f'{prop}')]),
             # validation_split = 0.2,
            callbacks=keras_callbacks)

""")
        f.close
        


## Tempalte 2 - Analysis of Hyper Parameter Grid Search

In [133]:
def python_grid_analysis(resample, fold, desc, prop):
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st"
    python_script=f"{desc}_{prop}_{resample}_{fold}_st.py"
    with open(f'{dir}/{python_script}','w') as f:
        f.write("""# Part one General and Data Hanlding

import numpy as np
import pandas as pd
import os
from pathlib import Path

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

# Part 2 Model

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import layers, models, initializers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger



# Data Block Functions

def path_fold(home,resample,i_fold):
    path="{}/CV/{}/fold_{}".format(os.getcwd(),resample,i_fold)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}/{i_fold}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def path_resample(home,resample):
    path="{}/CV/{}/".format(os.getcwd(),resample)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def cv_hp(df,home):
    resample_split  = ShuffleSplit(50, test_size=0.3, random_state=1)
    fold_split      = ShuffleSplit(5 , test_size=0.3, random_state=1)
    train_val_split = ShuffleSplit(1 , test_size=0.3, random_state=1)
    
    for resample, (train_val_index, test_index) in enumerate(resample_split.split(df)):
        train_val = pd.DataFrame(df['ID'].iloc[train_val_index])
        test = pd.DataFrame(df['ID'].iloc[test_index])
        for i, (train_index, val_index) in enumerate(train_val_split.split(train_val)):
            train = pd.DataFrame(df['ID'].iloc[train_index])
            val   = pd.DataFrame(df['ID'].iloc[val_index])
        resample_path = path_resample(home,resample)
        train.to_csv(f'{resample_path}/train.csv')
        val.to_csv(f'{resample_path}/val.csv')
        test.to_csv(f'{resample_path}/test.csv')
        # train,val,test to_csv
        for i_fold, (train_val_fold_index, test_fold_index) in enumerate(fold_split.split(train)):
            train_val_fold = pd.DataFrame(train['ID'].iloc[train_val_fold_index])
            test_fold = pd.DataFrame(train['ID'].iloc[test_fold_index])
            for i, (train_fold_index, val_fold_index) in enumerate(train_val_split.split(train_val_fold)):
                train_fold = pd.DataFrame(train_val_fold['ID'].iloc[train_fold_index])
                val_fold   = pd.DataFrame(train_val_fold['ID'].iloc[val_fold_index])
            i_fold_path = path_fold(home,resample,i_fold)
            train_fold.to_csv(f'{i_fold_path}/train.csv')
            val_fold.to_csv(f'{i_fold_path}/val.csv')
            test_fold.to_csv(f'{i_fold_path}/test.csv')
            

    return print("data organised into 50 CV with 5-fold inner CV")


# Accessing Data Via ID CV and Full table functions

def access_fold_csv(df,home,resample,fold):
    df_path = path_fold(home,resample,fold)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


def access_resample_csv(df,home,resample):
    df_path = path_resample(home,resample)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


# Loading X and Y Data Functions

def padding(X_descr_train_scaled):
#     Padding function so X data is always 250 dimensions
# Must be coupled with load_data. NB! double check if the scalling is not affected
# https://www.geeksforgeeks.org/python-call-function-from-another-function/
    a=X_descr_train_scaled.to_numpy()
    b=np.zeros((len(X_descr_train_scaled), 
                (250-int(X_descr_train_scaled.to_numpy().shape[1]))
               )
              )
    padded=np.concatenate((a,b),
                           axis=1, 
                          out=None, 
                          dtype=None
                         )
    return padded


def load_xy(file,desc):
    # Universal funciton for loading
# y_1, y_2, y_3, y_4 and x data from input csv (All, Train, Val or Train)
    y_1 = file[['dH']].copy()
    y_2 = file[['dS']].copy()
    y_3 = file[['dG']].copy()
    y_4 = file[['Tm']].copy()

    Y = file[['dH','dS','dG','Tm']].copy()
    # Convert y data into required input shape
    y_1 = y_1.to_numpy()
    y_1 = y_1.reshape(y_1.shape[0])
    y_2 = y_2.to_numpy()
    y_2 = y_2.reshape(y_2.shape[0])
    y_3 = y_3.to_numpy()
    y_3 = y_3.reshape(y_3.shape[0])
    y_4 = y_4.to_numpy()
    y_4 = y_4.reshape(y_4.shape[0])
    
    # Load features based on prop
    X = file[[col for col in file.columns if f'{desc}_'in col]]
    
    return y_1, y_2, y_3, y_4, Y, padding(X), X



# Model and Scoring Functions

def r2_func(y_true, y_pred, **kwargs):
    return metrics.r2_score(y_true, y_pred)
def rmse_func(y_true, y_pred, **kwargs):
    return np.sqrt(metrics.mean_squared_error(y_true, y_pred))  
def bias_func(y_true, y_pred, **kwargs):
    return np.mean(y_true-y_pred)
def sdep_func(y_true, y_pred, **kwargs):
    return (np.mean((y_true-y_pred-(np.mean(y_true-y_pred)))**2))**0.5
#these 4 are for tensorflow formats
def r2_func_tf(y_true, y_pred, **kwargs):
    numerator = tf.reduce_sum(tf.square(y_true - y_pred))
    denominator = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    r2 = 1 - numerator / denominator
    return r2
def rmse_func_tf(y_true, y_pred, **kwargs):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    rmse = tf.sqrt(mse)
    return rmse
def bias_func_tf(y_true, y_pred, **kwargs):
    bias = tf.reduce_mean(y_true - y_pred)
    return bias
def sdep_func_tf(y_true, y_pred, **kwargs):
    diff = y_true - y_pred
    mean_diff = tf.reduce_mean(diff)
    sdep = tf.sqrt(tf.reduce_mean(tf.square(diff - mean_diff)))
    return sdep

# Model


def build_model(hp):
    
#     Hyper parameters         
    model_type1 = hp.Choice("model_type1", ["CNN3","CNN2","CNN1"])
    model_type = hp.Choice("model_type", ["Dense3"])
 
    
#     INPUT for NN
    
    inputs = keras.Input(shape=(250,1))
    x_layer=inputs
    
#     MANDATORY CNN (optional to move into first condition hp.cond_scope
    # with hp.conditional_scope("model_type1", ["CNN0"]):
    #         if model_type1 == "CNN0":
    #             pass
#     CONDITIONAL CONVOLUTION LAYERS (Consider moving the above into CNN1) test 0-3 CNN and 0-3 Dense
    with hp.conditional_scope("model_type1", ["CNN1","CNN2""CNN3"]):
            if model_type1 != "CNN0":
                x_layer = keras.layers.Conv1D(32, 
                        kernel_size=(3), 
                        strides=(2), 
                        padding='valid', 
                        activation='relu', 
                        input_shape=(250,1),
                        name = 'conv1d_1'
                        )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = 'maxpooling_1')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = 'batchnorm_1')(x_layer)
                pass
                
            if model_type1 != "CNN1":
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_2'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_2')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_2')(x_layer)

            if model_type1 != "CNN1" or "CNN2":               
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_3'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_3')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_3')(x_layer)
                
#     FLATTEN AFTER CONVOLUTIONS
    x_layer = keras.layers.Flatten(name = 'flatten')(x_layer)
    
#     CONDITIONAL DENSE LAYERS
    # with hp.conditional_scope("model_type", ["Dense0"]):
    #     if model_type == "Dense0":
    #         pass
            
    with hp.conditional_scope("model_type", ["Dense3"]): #["Dense1","Dense2","Dense3"]
        if model_type != "Dense0":
            hp_layer_1= hp.Choice(f'layer_1', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_1,
                        activation='relu',
                        use_bias=True,
                        # name='layer_1',
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
        if model_type != "Dense1":
            hp_layer_2_2= hp.Choice(f'layer_2_2', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_2_2,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)

        if model_type != "Dense1" or "Dense2":
            hp_layer_3_3= hp.Choice(f'layer_3_3',  values=[16,32,64])
            
            x_layer = keras.layers.Dense(
                        hp_layer_3_3,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
#     OUTPUT LAYERS

    # output_1 = keras.layers.Dense(1, name='enthalpy_pred')(x_layer)
    # output_2 = keras.layers.Dense(1, name='entropy_pred')(x_layer)
    # output_3 = keras.layers.Dense(1, name='free_energy_pred')(x_layer)
    # output_4 = keras.layers.Dense(1, name='melting_temperature')(x_layer)

    # output_1 = keras.layers.Dense(1, name='dH')(x_layer)
    # output_2 = keras.layers.Dense(1, name='dS')(x_layer)
    # output_3 = keras.layers.Dense(1, name='dG')(x_layer)
    # output_4 = keras.layers.Dense(1, name='Tm')(x_layer)
    

    # model = Model(inputs=inputs, outputs=[output_1, output_2, output_3, output_4])
    output_1 = keras.layers.Dense(1, name='output')(x_layer)
    model = Model(inputs=inputs, outputs=output_1)
    
#     SETTINGS
#     SETTINGS

#     ADAPTIVE LEARNING RATE   
    
    initial_learning_rate = 0.01
    decay_steps = 10.0
    decay_rate = 0.5
    learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(
                                    initial_learning_rate, decay_steps, decay_rate)
    
#     SETTING ADAM OPTIMISER
    optimiser = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
    
#     COMPILE MODEl
    model.compile(loss = "mse" , 
                  optimizer = optimiser, 
                  metrics = ["mse",'mean_absolute_error',r2_func_tf, rmse_func_tf, bias_func_tf, sdep_func_tf])   
    
    return model
""")
        
        f.write(f"""              

# Code Execution 

df=pd.read_csv("Lomzov_dataset_IY.csv")
home=os.getcwd()
# data Generation
# cv_hp(df,home)

# Actual instrucitons
resample={resample}
fold={fold}
desc='{desc}'
prop = '{prop}' 
model_name = f"1DConv_st_{prop}" 
epochs = 300
batch  = 16

""")

        f.write("""
#  ANALYSIS SCRIPT 
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn.metrics import mean_absolute_error
import sys

def stats_hp(y_test_pred, Y_test, prop):
    y_test_np = Y_test[f'{prop}'].to_numpy()
    y_pred_test_np = y_test_pred

    r2_test = r2_score(y_test_np, y_pred_test_np)
    rmsd_test = (mean_squared_error(y_test_np, y_pred_test_np))**0.5
    bias_test = np.mean(y_pred_test_np - y_test_np)
    sdep_test = (np.mean((y_pred_test_np - y_test_np - bias_test)**2))**0.5
    r2 = '{:.3f}'.format(r2_test)
    rmsd = '{:.3f}'.format(rmsd_test)
    bias = '{:.3f}'.format(bias_test)
    sdep = '{:.3f}'.format(sdep_test)
    
    
    mse=mean_squared_error(y_test_np, y_pred_test_np)
    mse='{:.3f}'.format(mse)
    mae=mean_absolute_error(y_test_np, y_pred_test_np)
    mae='{:.3f}'.format(mae)
    try:
        a, b = np.polyfit(y_test_np, y_pred_test_np, 1)
        plot_a = '{:.3f}'.format(a[0])
        plot_b = '{:.3f}'.format(b[0])
    except np.linalg.LinAlgError:
        pass
    
    return r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae

def grid_analysis(df,home,resample,fold,prop, model_name):
    df2=pd.DataFrame()
    keyList = ['trial',
               'model_type1', 'model_type',
               'layer_3_3', 'layer_2_2', 'layer_1',
               f'r2_{prop}_{fold}',
               f'rmsd_{prop}_{fold}',
               f'bias_{prop}_{fold}',
               f'SDEP_{prop}_{fold}',
               f'gradient_{prop}_{fold}',
               f'b_{prop}_{fold}',
               f'mse_{prop}_{fold}',
               f'mae_{prop}_{fold}']
    n = dict(zip(keyList, [None]*len(keyList)))
    
    df_grid=pd.DataFrame(n,index=['a'])
   
    dir=path_fold(home, resample, fold)
    # model_name = f"1DConv_st_{prop}" 
    tunner_path =f"{dir}/{desc}/{model_name}/tunner"
    tuner = kt.GridSearch(build_model,
                           objective=kt.Objective('val_loss', 'min'),
                            # loss = 'val_loss',
                           # objective = ['val_mse','epoch_entropy_pred_mse','val_free_energy_pred_mse'],
                          directory=tunner_path,
                          overwrite=False,
                          project_name=f'{batch}')

    # Find all trial number and cycle through them
    selection=os.listdir(tuner.project_dir)
    for dir_ in selection:
        if "trial_" not in dir_:
            continue
        trial_number=str(dir_.replace("trial_",""))
        trial=tuner.oracle.get_trial(trial_number)
        model=tuner.load_model(trial)

        # train, val, test = access_resample_csv(df,home,resample)
        train_fold, val_fold, test_fold = access_fold_csv(df,home,resample,fold)
        
        y_1_train, y_2_train, y_3_train, y_4_train, Y_train, X_padded_train, X_train = load_xy(train_fold,desc)
        y_1_val,   y_2_val,   y_3_val,   y_4_val,   Y_val,   X_padded_val,   X_val   = load_xy(val_fold,desc)
        y_1_test,  y_2_test,  y_3_test,  y_4_test,  Y_test,  X_padded_test,  X_test  = load_xy(test_fold,desc)

        y_test_pred = model.predict(X_padded_test)
        
        r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae = stats_hp(y_test_pred, Y_test, prop)
        if trial_number in df_grid.index:
            n = df_grid.loc[trial_number].to_dict()
        n.update({"trial":trial_number,})
        n.update({f'r2_{prop}_{fold}':r2,f'rmsd_{prop}_{fold}': rmsd, 
                  f'bias_{prop}_{fold}': bias, f'SDEP_{prop}_{fold}': sdep,
                  f'gradient_{prop}_{fold}': plot_a, f'b_{prop}_{fold}': plot_b, 
                  f'mse_{prop}_{fold}':mse, f'mae_{prop}_{fold}':mae,})

        original_stdout = sys.stdout 	
        with open(f'{prop}_{fold}_{resample}.txt', 'w') as f:
            sys.stdout = f
            trial.display_hyperparameters()
            # Reset the standard output
            sys.stdout = original_stdout 
        f.close
        data = open(f'{prop}_{fold}_{resample}.txt', 'r').read()
        data_hyp_param=data.replace(': ','\\n').split('\\n')
        i_index=0
        while i_index+1 < len(data_hyp_param):
            n.update({data_hyp_param[i_index]:data_hyp_param[i_index+1]})
            i_index+=2
        os.system(f'rm {prop}_{fold}_{resample}.txt')

        df_grid.loc[f'{trial_number}']=n
    df_grid.to_csv(f'{dir}/{desc}/{model_name}/{prop}_{fold}.csv')
    # df_grid.to_csv(f'{prop}_{fold}.csv')
    return

grid_analysis(df,home,resample,fold,prop,model_name)

""")
        f.close
        


## Template 3 - Create Archive and Delete folders

In [195]:
def python_archive_del_hp(resample, fold, desc, prop):
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st_arch"
    python_script=f"{job_name}.py"
    with open(f'{dir}/{python_script}','w') as f:
        f.write("""# Part one General and Data Hanlding

import tarfile
import os
from pathlib import Path

def path_fold(home,resample,i_fold):
    path="{}/CV/{}/fold_{}".format(os.getcwd(),resample,i_fold)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}/{i_fold}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path
""")
        
        f.write(f"""              

# Code Execution 

# df=pd.read_csv("Lomzov_dataset_IY.csv")     # Not required for creating archive and delteing Folders
home=os.getcwd()
# data Generation
# cv_hp(df,home)

# Actual instrucitons
resample={resample}
fold={fold}
desc='{desc}'
prop = '{prop}' 
model_name = f"1DConv_st_{prop}" 
epochs = 300
batch  = 16

""")

        f.write("""
import tarfile
import os
from pathlib import Path
#  Create Archive and Delete Folders

def create_tar_archive(work_dir,files):
    # Create a tar.gz archive
    with tarfile.open(f"{work_dir}tuner_results.tar.gz", "w:gz") as archive:
        for file in files:
            archive.add(f"{work_dir}{file}", arcname=os.path.basename(f"{file}"))
        archive.close()

        
def archive_hyper_params(home,resample,fold,prop,desc,model_name):
    # prop not required
    dir=path_fold(home, resample, fold)
    folder_dir=f'{dir}/{desc}/{model_name}/'
    folders_to_archive=os.listdir(folder_dir)
    create_tar_archive(folder_dir, folders_to_archive)
    
    return

def delete_folders_after_archive(home,resample,fold,prop,desc,model_name):
    # prop not required
    dir=path_fold(home, resample, fold)
    folder_dir=f'{dir}/{desc}/{model_name}/'
    folders_to_archive=os.listdir(folder_dir)
    folders_to_archive = list(filter(lambda i:'.' not in i, folders_to_archive))
    for folder in folders_to_archive:
        os.system(f'rm -r {folder_dir}{folder}')

    return
    
archive_hyper_params(home,resample,fold,prop,desc,model_name)
delete_folders_after_archive(home,resample,fold,prop,desc,model_name)

""")
        f.close
        


## Template 2+3

In [122]:
def python_grid_analysis_archive(resample, fold, desc, prop):
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st"
    python_script=f"{desc}_{prop}_{resample}_{fold}_st.py"
    with open(f'{dir}/{python_script}','w') as f:
        f.write("""# Part one General and Data Hanlding

import numpy as np
import pandas as pd
import os
from pathlib import Path

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

# Part 2 Model

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import layers, models, initializers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger



# Data Block Functions

def path_fold(home,resample,i_fold):
    path="{}/CV/{}/fold_{}".format(os.getcwd(),resample,i_fold)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}/{i_fold}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def path_resample(home,resample):
    path="{}/CV/{}/".format(os.getcwd(),resample)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def cv_hp(df,home):
    resample_split  = ShuffleSplit(50, test_size=0.3, random_state=1)
    fold_split      = ShuffleSplit(5 , test_size=0.3, random_state=1)
    train_val_split = ShuffleSplit(1 , test_size=0.3, random_state=1)
    
    for resample, (train_val_index, test_index) in enumerate(resample_split.split(df)):
        train_val = pd.DataFrame(df['ID'].iloc[train_val_index])
        test = pd.DataFrame(df['ID'].iloc[test_index])
        for i, (train_index, val_index) in enumerate(train_val_split.split(train_val)):
            train = pd.DataFrame(df['ID'].iloc[train_index])
            val   = pd.DataFrame(df['ID'].iloc[val_index])
        resample_path = path_resample(home,resample)
        train.to_csv(f'{resample_path}/train.csv')
        val.to_csv(f'{resample_path}/val.csv')
        test.to_csv(f'{resample_path}/test.csv')
        # train,val,test to_csv
        for i_fold, (train_val_fold_index, test_fold_index) in enumerate(fold_split.split(train)):
            train_val_fold = pd.DataFrame(train['ID'].iloc[train_val_fold_index])
            test_fold = pd.DataFrame(train['ID'].iloc[test_fold_index])
            for i, (train_fold_index, val_fold_index) in enumerate(train_val_split.split(train_val_fold)):
                train_fold = pd.DataFrame(train_val_fold['ID'].iloc[train_fold_index])
                val_fold   = pd.DataFrame(train_val_fold['ID'].iloc[val_fold_index])
            i_fold_path = path_fold(home,resample,i_fold)
            train_fold.to_csv(f'{i_fold_path}/train.csv')
            val_fold.to_csv(f'{i_fold_path}/val.csv')
            test_fold.to_csv(f'{i_fold_path}/test.csv')
            

    return print("data organised into 50 CV with 5-fold inner CV")


# Accessing Data Via ID CV and Full table functions

def access_fold_csv(df,home,resample,fold):
    df_path = path_fold(home,resample,fold)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


def access_resample_csv(df,home,resample):
    df_path = path_resample(home,resample)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


# Loading X and Y Data Functions

def padding(X_descr_train_scaled):
#     Padding function so X data is always 250 dimensions
# Must be coupled with load_data. NB! double check if the scalling is not affected
# https://www.geeksforgeeks.org/python-call-function-from-another-function/
    a=X_descr_train_scaled.to_numpy()
    b=np.zeros((len(X_descr_train_scaled), 
                (250-int(X_descr_train_scaled.to_numpy().shape[1]))
               )
              )
    padded=np.concatenate((a,b),
                           axis=1, 
                          out=None, 
                          dtype=None
                         )
    return padded


def load_xy(file,desc):
    # Universal funciton for loading
# y_1, y_2, y_3, y_4 and x data from input csv (All, Train, Val or Train)
    y_1 = file[['dH']].copy()
    y_2 = file[['dS']].copy()
    y_3 = file[['dG']].copy()
    y_4 = file[['Tm']].copy()

    Y = file[['dH','dS','dG','Tm']].copy()
    # Convert y data into required input shape
    y_1 = y_1.to_numpy()
    y_1 = y_1.reshape(y_1.shape[0])
    y_2 = y_2.to_numpy()
    y_2 = y_2.reshape(y_2.shape[0])
    y_3 = y_3.to_numpy()
    y_3 = y_3.reshape(y_3.shape[0])
    y_4 = y_4.to_numpy()
    y_4 = y_4.reshape(y_4.shape[0])
    
    # Load features based on prop
    X = file[[col for col in file.columns if f'{desc}_'in col]]
    
    return y_1, y_2, y_3, y_4, Y, padding(X), X



# Model and Scoring Functions

def r2_func(y_true, y_pred, **kwargs):
    return metrics.r2_score(y_true, y_pred)
def rmse_func(y_true, y_pred, **kwargs):
    return np.sqrt(metrics.mean_squared_error(y_true, y_pred))  
def bias_func(y_true, y_pred, **kwargs):
    return np.mean(y_true-y_pred)
def sdep_func(y_true, y_pred, **kwargs):
    return (np.mean((y_true-y_pred-(np.mean(y_true-y_pred)))**2))**0.5
#these 4 are for tensorflow formats
def r2_func_tf(y_true, y_pred, **kwargs):
    numerator = tf.reduce_sum(tf.square(y_true - y_pred))
    denominator = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    r2 = 1 - numerator / denominator
    return r2
def rmse_func_tf(y_true, y_pred, **kwargs):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    rmse = tf.sqrt(mse)
    return rmse
def bias_func_tf(y_true, y_pred, **kwargs):
    bias = tf.reduce_mean(y_true - y_pred)
    return bias
def sdep_func_tf(y_true, y_pred, **kwargs):
    diff = y_true - y_pred
    mean_diff = tf.reduce_mean(diff)
    sdep = tf.sqrt(tf.reduce_mean(tf.square(diff - mean_diff)))
    return sdep

# Model


def build_model(hp):
    
#     Hyper parameters         
    model_type1 = hp.Choice("model_type1", ["CNN3","CNN2","CNN1"])
    model_type = hp.Choice("model_type", ["Dense3"])
 
    
#     INPUT for NN
    
    inputs = keras.Input(shape=(250,1))
    x_layer=inputs
    
#     MANDATORY CNN (optional to move into first condition hp.cond_scope
    # with hp.conditional_scope("model_type1", ["CNN0"]):
    #         if model_type1 == "CNN0":
    #             pass
#     CONDITIONAL CONVOLUTION LAYERS (Consider moving the above into CNN1) test 0-3 CNN and 0-3 Dense
    with hp.conditional_scope("model_type1", ["CNN1","CNN2""CNN3"]):
            if model_type1 != "CNN0":
                x_layer = keras.layers.Conv1D(32, 
                        kernel_size=(3), 
                        strides=(2), 
                        padding='valid', 
                        activation='relu', 
                        input_shape=(250,1),
                        name = 'conv1d_1'
                        )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = 'maxpooling_1')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = 'batchnorm_1')(x_layer)
                pass
                
            if model_type1 != "CNN1":
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_2'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_2')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_2')(x_layer)

            if model_type1 != "CNN1" or "CNN2":               
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_3'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_3')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_3')(x_layer)
                
#     FLATTEN AFTER CONVOLUTIONS
    x_layer = keras.layers.Flatten(name = 'flatten')(x_layer)
    
#     CONDITIONAL DENSE LAYERS
    # with hp.conditional_scope("model_type", ["Dense0"]):
    #     if model_type == "Dense0":
    #         pass
            
    with hp.conditional_scope("model_type", ["Dense3"]): #["Dense1","Dense2","Dense3"]
        if model_type != "Dense0":
            hp_layer_1= hp.Choice(f'layer_1', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_1,
                        activation='relu',
                        use_bias=True,
                        # name='layer_1',
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
        if model_type != "Dense1":
            hp_layer_2_2= hp.Choice(f'layer_2_2', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_2_2,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)

        if model_type != "Dense1" or "Dense2":
            hp_layer_3_3= hp.Choice(f'layer_3_3',  values=[16,32,64])
            
            x_layer = keras.layers.Dense(
                        hp_layer_3_3,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
#     OUTPUT LAYERS

    # output_1 = keras.layers.Dense(1, name='enthalpy_pred')(x_layer)
    # output_2 = keras.layers.Dense(1, name='entropy_pred')(x_layer)
    # output_3 = keras.layers.Dense(1, name='free_energy_pred')(x_layer)
    # output_4 = keras.layers.Dense(1, name='melting_temperature')(x_layer)

    # output_1 = keras.layers.Dense(1, name='dH')(x_layer)
    # output_2 = keras.layers.Dense(1, name='dS')(x_layer)
    # output_3 = keras.layers.Dense(1, name='dG')(x_layer)
    # output_4 = keras.layers.Dense(1, name='Tm')(x_layer)
    

    # model = Model(inputs=inputs, outputs=[output_1, output_2, output_3, output_4])
    output_1 = keras.layers.Dense(1, name='output')(x_layer)
    model = Model(inputs=inputs, outputs=output_1)
    
#     SETTINGS
#     SETTINGS

#     ADAPTIVE LEARNING RATE   
    
    initial_learning_rate = 0.01
    decay_steps = 10.0
    decay_rate = 0.5
    learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(
                                    initial_learning_rate, decay_steps, decay_rate)
    
#     SETTING ADAM OPTIMISER
    optimiser = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
    
#     COMPILE MODEl
    model.compile(loss = "mse" , 
                  optimizer = optimiser, 
                  metrics = ["mse",'mean_absolute_error',r2_func_tf, rmse_func_tf, bias_func_tf, sdep_func_tf])   
    
    return model
""")
        
        f.write(f"""              

# Code Execution 

df=pd.read_csv("Lomzov_dataset_IY.csv")
home=os.getcwd()
# data Generation
# cv_hp(df,home)

# Actual instrucitons
resample={resample}
fold={fold}
desc='{desc}'
prop = '{prop}' 
model_name = f"1DConv_st_{prop}" 
epochs = 300
batch  = 16

""")

        f.write("""
#  ANALYSIS SCRIPT 
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn.metrics import mean_absolute_error
import sys

def stats_hp(y_test_pred, Y_test, prop):
    y_test_np = Y_test[f'{prop}'].to_numpy()
    y_pred_test_np = y_test_pred

    r2_test = r2_score(y_test_np, y_pred_test_np)
    rmsd_test = (mean_squared_error(y_test_np, y_pred_test_np))**0.5
    bias_test = np.mean(y_pred_test_np - y_test_np)
    sdep_test = (np.mean((y_pred_test_np - y_test_np - bias_test)**2))**0.5
    r2 = '{:.3f}'.format(r2_test)
    rmsd = '{:.3f}'.format(rmsd_test)
    bias = '{:.3f}'.format(bias_test)
    sdep = '{:.3f}'.format(sdep_test)
    
    
    mse=mean_squared_error(y_test_np, y_pred_test_np)
    mse='{:.3f}'.format(mse)
    mae=mean_absolute_error(y_test_np, y_pred_test_np)
    mae='{:.3f}'.format(mae)
    try:
        a, b = np.polyfit(y_test_np, y_pred_test_np, 1)
        plot_a = '{:.3f}'.format(a[0])
        plot_b = '{:.3f}'.format(b[0])
    except np.linalg.LinAlgError:
        pass
    
    return r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae

def grid_analysis(df,home,resample,fold,prop, model_name):
    df2=pd.DataFrame()
    keyList = ['trial',
               'model_type1', 'model_type',
               'layer_3_3', 'layer_2_2', 'layer_1',
               f'r2_{prop}_{fold}',
               f'rmsd_{prop}_{fold}',
               f'bias_{prop}_{fold}',
               f'SDEP_{prop}_{fold}',
               f'gradient_{prop}_{fold}',
               f'b_{prop}_{fold}',
               f'mse_{prop}_{fold}',
               f'mae_{prop}_{fold}']
    n = dict(zip(keyList, [None]*len(keyList)))
    
    df_grid=pd.DataFrame(n,index=['a'])
   
    dir=path_fold(home, resample, fold)
    # model_name = f"1DConv_st_{prop}" 
    tunner_path =f"{dir}/{desc}/{model_name}/tunner"
    tuner = kt.GridSearch(build_model,
                           objective=kt.Objective('val_loss', 'min'),
                            # loss = 'val_loss',
                           # objective = ['val_mse','epoch_entropy_pred_mse','val_free_energy_pred_mse'],
                          directory=tunner_path,
                          overwrite=False,
                          project_name=f'{batch}')

    # Find all trial number and cycle through them
    selection=os.listdir(tuner.project_dir)
    for dir_ in selection:
        if "trial_" not in dir_:
            continue
        trial_number=str(dir_.replace("trial_",""))
        trial=tuner.oracle.get_trial(trial_number)
        model=tuner.load_model(trial)

        # train, val, test = access_resample_csv(df,home,resample)
        train_fold, val_fold, test_fold = access_fold_csv(df,home,resample,fold)
        
        y_1_train, y_2_train, y_3_train, y_4_train, Y_train, X_padded_train, X_train = load_xy(train_fold,desc)
        y_1_val,   y_2_val,   y_3_val,   y_4_val,   Y_val,   X_padded_val,   X_val   = load_xy(val_fold,desc)
        y_1_test,  y_2_test,  y_3_test,  y_4_test,  Y_test,  X_padded_test,  X_test  = load_xy(test_fold,desc)

        y_test_pred = model.predict(X_padded_test)
        
        r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae = stats_hp(y_test_pred, Y_test, prop)
        if trial_number in df_grid.index:
            n = df_grid.loc[trial_number].to_dict()
        n.update({"trial":trial_number,})
        n.update({f'r2_{prop}_{fold}':r2,f'rmsd_{prop}_{fold}': rmsd, 
                  f'bias_{prop}_{fold}': bias, f'SDEP_{prop}_{fold}': sdep,
                  f'gradient_{prop}_{fold}': plot_a, f'b_{prop}_{fold}': plot_b, 
                  f'mse_{prop}_{fold}':mse, f'mae_{prop}_{fold}':mae,})

        original_stdout = sys.stdout 	
        with open(f'{prop}_{fold}_{resample}.txt', 'w') as f:
            sys.stdout = f
            trial.display_hyperparameters()
            # Reset the standard output
            sys.stdout = original_stdout 
        f.close
        data = open(f'{prop}_{fold}_{resample}.txt', 'r').read()
        data_hyp_param=data.replace(': ','\\n').split('\\n')
        i_index=0
        while i_index+1 < len(data_hyp_param):
            n.update({data_hyp_param[i_index]:data_hyp_param[i_index+1]})
            i_index+=2
        os.system(f'rm {prop}_{fold}_{resample}.txt')

        df_grid.loc[f'{trial_number}']=n
    df_grid.to_csv(f'{dir}/{desc}/{model_name}/{prop}_{fold}.csv')
    # df_grid.to_csv(f'{prop}_{fold}.csv')
    return

grid_analysis(df,home,resample,fold,prop,model_name)

""")
        f.close
        


## Template 4 - Access 5 fold in a resample. Load the prop_fold.csv file. find best hyper paramter set and train model_final. Then Make predicitons and store CSV (True vs pred and Stats)

In [159]:
def python_temp_4(resample, desc, prop):
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_st"
    python_script=f"{desc}_{prop}_{resample}_st.py"
    with open(f'{dir}/{python_script}','w') as f:
        f.write("""# Part one General and Data Hanlding

import numpy as np
import pandas as pd
import os
from pathlib import Path

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

# Part 2 Model

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import layers, models, initializers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger



# Data Block Functions

def path_fold(home,resample,i_fold):
    path="{}/CV/{}/fold_{}".format(os.getcwd(),resample,i_fold)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}/{i_fold}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def path_resample(home,resample):
    path="{}/CV/{}/".format(os.getcwd(),resample)
        
    # Define the directory path
    directory_path = Path(f"{home}/CV/{resample}")
    
    # Ensure the directory exists, create it if necessary
    directory_path.mkdir(parents=True, exist_ok=True)

    return directory_path

def cv_hp(df,home):
    resample_split  = ShuffleSplit(50, test_size=0.3, random_state=1)
    fold_split      = ShuffleSplit(5 , test_size=0.3, random_state=1)
    train_val_split = ShuffleSplit(1 , test_size=0.3, random_state=1)
    
    for resample, (train_val_index, test_index) in enumerate(resample_split.split(df)):
        train_val = pd.DataFrame(df['ID'].iloc[train_val_index])
        test = pd.DataFrame(df['ID'].iloc[test_index])
        for i, (train_index, val_index) in enumerate(train_val_split.split(train_val)):
            train = pd.DataFrame(df['ID'].iloc[train_index])
            val   = pd.DataFrame(df['ID'].iloc[val_index])
        resample_path = path_resample(home,resample)
        train.to_csv(f'{resample_path}/train.csv')
        val.to_csv(f'{resample_path}/val.csv')
        test.to_csv(f'{resample_path}/test.csv')
        # train,val,test to_csv
        for i_fold, (train_val_fold_index, test_fold_index) in enumerate(fold_split.split(train)):
            train_val_fold = pd.DataFrame(train['ID'].iloc[train_val_fold_index])
            test_fold = pd.DataFrame(train['ID'].iloc[test_fold_index])
            for i, (train_fold_index, val_fold_index) in enumerate(train_val_split.split(train_val_fold)):
                train_fold = pd.DataFrame(train_val_fold['ID'].iloc[train_fold_index])
                val_fold   = pd.DataFrame(train_val_fold['ID'].iloc[val_fold_index])
            i_fold_path = path_fold(home,resample,i_fold)
            train_fold.to_csv(f'{i_fold_path}/train.csv')
            val_fold.to_csv(f'{i_fold_path}/val.csv')
            test_fold.to_csv(f'{i_fold_path}/test.csv')
            

    return print("data organised into 50 CV with 5-fold inner CV")


# Accessing Data Via ID CV and Full table functions

def access_fold_csv(df,home,resample,fold):
    df_path = path_fold(home,resample,fold)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


def access_resample_csv(df,home,resample):
    df_path = path_resample(home,resample)
    train_df=pd.read_csv(f'{df_path}/train.csv')
    val_df=pd.read_csv(f'{df_path}/val.csv')
    test_df=pd.read_csv(f'{df_path}/test.csv')

    train_df=df[df["ID"].isin(train_df['ID'])]
    val_df=df[df["ID"].isin(val_df['ID'])]
    test_df=df[df["ID"].isin(test_df['ID'])]
    return train_df, val_df, test_df


# Loading X and Y Data Functions

def padding(X_descr_train_scaled):
#     Padding function so X data is always 250 dimensions
# Must be coupled with load_data. NB! double check if the scalling is not affected
# https://www.geeksforgeeks.org/python-call-function-from-another-function/
    a=X_descr_train_scaled.to_numpy()
    b=np.zeros((len(X_descr_train_scaled), 
                (250-int(X_descr_train_scaled.to_numpy().shape[1]))
               )
              )
    padded=np.concatenate((a,b),
                           axis=1, 
                          out=None, 
                          dtype=None
                         )
    return padded


def load_xy(file,desc):
    # Universal funciton for loading
# y_1, y_2, y_3, y_4 and x data from input csv (All, Train, Val or Train)
    y_1 = file[['dH']].copy()
    y_2 = file[['dS']].copy()
    y_3 = file[['dG']].copy()
    y_4 = file[['Tm']].copy()

    Y = file[['dH','dS','dG','Tm']].copy()
    # Convert y data into required input shape
    y_1 = y_1.to_numpy()
    y_1 = y_1.reshape(y_1.shape[0])
    y_2 = y_2.to_numpy()
    y_2 = y_2.reshape(y_2.shape[0])
    y_3 = y_3.to_numpy()
    y_3 = y_3.reshape(y_3.shape[0])
    y_4 = y_4.to_numpy()
    y_4 = y_4.reshape(y_4.shape[0])
    
    # Load features based on prop
    X = file[[col for col in file.columns if f'{desc}_'in col]]
    
    return y_1, y_2, y_3, y_4, Y, padding(X), X



# Model and Scoring Functions

def r2_func(y_true, y_pred, **kwargs):
    return metrics.r2_score(y_true, y_pred)
def rmse_func(y_true, y_pred, **kwargs):
    return np.sqrt(metrics.mean_squared_error(y_true, y_pred))  
def bias_func(y_true, y_pred, **kwargs):
    return np.mean(y_true-y_pred)
def sdep_func(y_true, y_pred, **kwargs):
    return (np.mean((y_true-y_pred-(np.mean(y_true-y_pred)))**2))**0.5
#these 4 are for tensorflow formats
def r2_func_tf(y_true, y_pred, **kwargs):
    numerator = tf.reduce_sum(tf.square(y_true - y_pred))
    denominator = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    r2 = 1 - numerator / denominator
    return r2
def rmse_func_tf(y_true, y_pred, **kwargs):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    rmse = tf.sqrt(mse)
    return rmse
def bias_func_tf(y_true, y_pred, **kwargs):
    bias = tf.reduce_mean(y_true - y_pred)
    return bias
def sdep_func_tf(y_true, y_pred, **kwargs):
    diff = y_true - y_pred
    mean_diff = tf.reduce_mean(diff)
    sdep = tf.sqrt(tf.reduce_mean(tf.square(diff - mean_diff)))
    return sdep

# Model


def build_model(hp):
    
#     Hyper parameters         
    model_type1 = hp.Choice("model_type1", ["CNN3","CNN2","CNN1"])
    model_type = hp.Choice("model_type", ["Dense3"])
 
    
#     INPUT for NN
    
    inputs = keras.Input(shape=(250,1))
    x_layer=inputs
    
#     MANDATORY CNN (optional to move into first condition hp.cond_scope
    # with hp.conditional_scope("model_type1", ["CNN0"]):
    #         if model_type1 == "CNN0":
    #             pass
#     CONDITIONAL CONVOLUTION LAYERS (Consider moving the above into CNN1) test 0-3 CNN and 0-3 Dense
    with hp.conditional_scope("model_type1", ["CNN1","CNN2""CNN3"]):
            if model_type1 != "CNN0":
                x_layer = keras.layers.Conv1D(32, 
                        kernel_size=(3), 
                        strides=(2), 
                        padding='valid', 
                        activation='relu', 
                        input_shape=(250,1),
                        name = 'conv1d_1'
                        )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = 'maxpooling_1')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = 'batchnorm_1')(x_layer)
                pass
                
            if model_type1 != "CNN1":
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_2'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_2')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_2')(x_layer)

            if model_type1 != "CNN1" or "CNN2":               
                x_layer = keras.layers.Conv1D(32, 
                                    kernel_size=(3), 
                                    strides=(2), 
                                    padding='valid', 
                                    activation='relu', 
                                    name = f'conv1d_3'
                                    )(x_layer)
                x_layer = keras.layers.MaxPooling1D((2), name = f'maxpooling_3')(x_layer)
                x_layer = keras.layers.BatchNormalization(name = f'batchnorm_3')(x_layer)
                
#     FLATTEN AFTER CONVOLUTIONS
    x_layer = keras.layers.Flatten(name = 'flatten')(x_layer)
    
#     CONDITIONAL DENSE LAYERS
    # with hp.conditional_scope("model_type", ["Dense0"]):
    #     if model_type == "Dense0":
    #         pass
            
    with hp.conditional_scope("model_type", ["Dense3"]): #["Dense1","Dense2","Dense3"]
        if model_type != "Dense0":
            hp_layer_1= hp.Choice(f'layer_1', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_1,
                        activation='relu',
                        use_bias=True,
                        # name='layer_1',
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
        if model_type != "Dense1":
            hp_layer_2_2= hp.Choice(f'layer_2_2', values=[16,32,64,128])

            x_layer = keras.layers.Dense(
                        hp_layer_2_2,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)

        if model_type != "Dense1" or "Dense2":
            hp_layer_3_3= hp.Choice(f'layer_3_3',  values=[16,32,64])
            
            x_layer = keras.layers.Dense(
                        hp_layer_3_3,
                        activation='relu',
                        use_bias=True,
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        kernel_regularizer=None,
                        bias_regularizer=None,
                        activity_regularizer=None,
                        kernel_constraint=None,
                        bias_constraint=None
                    )(x_layer)
#     OUTPUT LAYERS

    # output_1 = keras.layers.Dense(1, name='enthalpy_pred')(x_layer)
    # output_2 = keras.layers.Dense(1, name='entropy_pred')(x_layer)
    # output_3 = keras.layers.Dense(1, name='free_energy_pred')(x_layer)
    # output_4 = keras.layers.Dense(1, name='melting_temperature')(x_layer)

    # output_1 = keras.layers.Dense(1, name='dH')(x_layer)
    # output_2 = keras.layers.Dense(1, name='dS')(x_layer)
    # output_3 = keras.layers.Dense(1, name='dG')(x_layer)
    # output_4 = keras.layers.Dense(1, name='Tm')(x_layer)
    

    # model = Model(inputs=inputs, outputs=[output_1, output_2, output_3, output_4])
    output_1 = keras.layers.Dense(1, name='output')(x_layer)
    model = Model(inputs=inputs, outputs=output_1)
    
#     SETTINGS
#     SETTINGS

#     ADAPTIVE LEARNING RATE   
    
    initial_learning_rate = 0.01
    decay_steps = 10.0
    decay_rate = 0.5
    learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(
                                    initial_learning_rate, decay_steps, decay_rate)
    
#     SETTING ADAM OPTIMISER
    optimiser = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
    
#     COMPILE MODEl
    model.compile(loss = "mse" , 
                  optimizer = optimiser, 
                  metrics = ["mse",'mean_absolute_error',r2_func_tf, rmse_func_tf, bias_func_tf, sdep_func_tf])   
    
    return model
""")
        
        f.write(f"""              

# Code Execution 

df=pd.read_csv("Lomzov_dataset_IY.csv")
home=os.getcwd()
# data Generation
# cv_hp(df,home)

# Actual instrucitons
GSHT_list=['dH','dS','dG','Tm']
resample={resample}
fold=5
desc='{desc}'
prop = '{prop}' 
model_name = f"1DConv_st_{prop}" 
epochs = 300
batch  = 16

""")

        f.write("""
#  ANALYSIS SCRIPT 
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn.metrics import mean_absolute_error
import sys

def stats_hp(y_test_pred, Y_test, prop):
    y_test_np = Y_test[f'{prop}'].to_numpy()
    y_pred_test_np = y_test_pred

    r2_test = r2_score(y_test_np, y_pred_test_np)
    rmsd_test = (mean_squared_error(y_test_np, y_pred_test_np))**0.5
    bias_test = np.mean(y_pred_test_np - y_test_np)
    sdep_test = (np.mean((y_pred_test_np - y_test_np - bias_test)**2))**0.5
    r2 = '{:.3f}'.format(r2_test)
    rmsd = '{:.3f}'.format(rmsd_test)
    bias = '{:.3f}'.format(bias_test)
    sdep = '{:.3f}'.format(sdep_test)
    
    
    mse=mean_squared_error(y_test_np, y_pred_test_np)
    mse='{:.3f}'.format(mse)
    mae=mean_absolute_error(y_test_np, y_pred_test_np)
    mae='{:.3f}'.format(mae)
    try:
        a, b = np.polyfit(y_test_np, y_pred_test_np, 1)
        plot_a = '{:.3f}'.format(a[0])
        plot_b = '{:.3f}'.format(b[0])
    except np.linalg.LinAlgError:
        pass
    
    return r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae

def grid_analysis(df,home,resample,fold,prop, model_name):
    df2=pd.DataFrame()
    keyList = ['trial',
               'model_type1', 'model_type',
               'layer_3_3', 'layer_2_2', 'layer_1',
               f'r2_{prop}_{fold}',
               f'rmsd_{prop}_{fold}',
               f'bias_{prop}_{fold}',
               f'SDEP_{prop}_{fold}',
               f'gradient_{prop}_{fold}',
               f'b_{prop}_{fold}',
               f'mse_{prop}_{fold}',
               f'mae_{prop}_{fold}']
    n = dict(zip(keyList, [None]*len(keyList)))
    
    df_grid=pd.DataFrame(n,index=['a'])
   
    dir=path_fold(home, resample, fold)
    # model_name = f"1DConv_st_{prop}" 
    tunner_path =f"{dir}/{desc}/{model_name}/tunner"
    tuner = kt.GridSearch(build_model,
                           objective=kt.Objective('val_loss', 'min'),
                            # loss = 'val_loss',
                           # objective = ['val_mse','epoch_entropy_pred_mse','val_free_energy_pred_mse'],
                          directory=tunner_path,
                          overwrite=False,
                          project_name=f'{batch}')

    # Find all trial number and cycle through them
    selection=os.listdir(tuner.project_dir)
    for dir_ in selection:
        if "trial_" not in dir_:
            continue
        trial_number=str(dir_.replace("trial_",""))
        trial=tuner.oracle.get_trial(trial_number)
        model=tuner.load_model(trial)

        # train, val, test = access_resample_csv(df,home,resample)
        train_fold, val_fold, test_fold = access_fold_csv(df,home,resample,fold)
        
        y_1_train, y_2_train, y_3_train, y_4_train, Y_train, X_padded_train, X_train = load_xy(train_fold,desc)
        y_1_val,   y_2_val,   y_3_val,   y_4_val,   Y_val,   X_padded_val,   X_val   = load_xy(val_fold,desc)
        y_1_test,  y_2_test,  y_3_test,  y_4_test,  Y_test,  X_padded_test,  X_test  = load_xy(test_fold,desc)

        y_test_pred = model.predict(X_padded_test)
        
        r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae = stats_hp(y_test_pred, Y_test, prop)
        if trial_number in df_grid.index:
            n = df_grid.loc[trial_number].to_dict()
        n.update({"trial":trial_number,})
        n.update({f'r2_{prop}_{fold}':r2,f'rmsd_{prop}_{fold}': rmsd, 
                  f'bias_{prop}_{fold}': bias, f'SDEP_{prop}_{fold}': sdep,
                  f'gradient_{prop}_{fold}': plot_a, f'b_{prop}_{fold}': plot_b, 
                  f'mse_{prop}_{fold}':mse, f'mae_{prop}_{fold}':mae,})

        original_stdout = sys.stdout 	
        with open(f'{prop}_{fold}_{resample}.txt', 'w') as f:
            sys.stdout = f
            trial.display_hyperparameters()
            # Reset the standard output
            sys.stdout = original_stdout 
        f.close
        data = open(f'{prop}_{fold}_{resample}.txt', 'r').read()
        data_hyp_param=data.replace(': ','\\n').split('\\n')
        i_index=0
        while i_index+1 < len(data_hyp_param):
            n.update({data_hyp_param[i_index]:data_hyp_param[i_index+1]})
            i_index+=2
        os.system(f'rm {prop}_{fold}_{resample}.txt')

        df_grid.loc[f'{trial_number}']=n
    df_grid.to_csv(f'{dir}/{desc}/{model_name}/{prop}_{fold}.csv')
    # df_grid.to_csv(f'{prop}_{fold}.csv')
    return
# No longer needed this execution
# grid_analysis(df,home,resample,fold,prop,model_name)

####### Template for Archiving and Deleting Files

import tarfile
import os
from pathlib import Path
#  Create Archive and Delete Folders

def create_tar_archive(work_dir,files):
    # Create a tar.gz archive
    with tarfile.open(f"{work_dir}tuner_results.tar.gz", "w:gz") as archive:
        for file in files:
            archive.add(f"{work_dir}{file}", arcname=os.path.basename(f"{file}"))
        archive.close()

        
def archive_hyper_params(home,resample,fold,prop,desc,model_name):
    # prop not required
    dir=path_fold(home, resample, fold)
    folder_dir=f'{dir}/{desc}/{model_name}/'
    folders_to_archive=os.listdir(folder_dir)
    create_tar_archive(folder_dir, folders_to_archive)
    
    return

def delete_folders_after_archive(home,resample,fold,prop,desc,model_name):
    # prop not required
    dir=path_fold(home, resample, fold)
    folder_dir=f'{dir}/{desc}/{model_name}/'
    folders_to_archive=os.listdir(folder_dir)
    folders_to_archive = list(filter(lambda i:'.' not in i, folders_to_archive))
    for folder in folders_to_archive:
        os.system(f'rm -r {folder_dir}{folder}')

    return
# no longer need execution  
# archive_hyper_params(home,resample,fold,prop,desc,model_name)
# delete_folders_after_archive(home,resample,fold,prop,desc,model_name)

# ######## Find best hyper params and train and evaluation on resample data
train, val, test = access_resample_csv(df,home,resample)

y_1_train, y_2_train, y_3_train, y_4_train, Y_train, X_padded_train, X_train = load_xy(train,desc)
y_1_val,   y_2_val,   y_3_val,   y_4_val,   Y_val,   X_padded_val,   X_val   = load_xy(val,desc)
y_1_test,  y_2_test,  y_3_test,  y_4_test,  Y_test,  X_padded_test,  X_test  = load_xy(test,desc)

fold_path = path_fold(home,resample,fold)
resample_path = path_resample(home,resample)

df_combine = pd.DataFrame()
df_summary = pd.DataFrame()
df_combine_all = pd.DataFrame()

# for resample in range(1):
df_combine_all = pd.DataFrame()
df_summary = pd.DataFrame()
# for prop in GSHT_list:
#     print(prop)
for fold in range(5):
    dir = path_fold(home, resample, fold)
    model_name = f"1DConv_st_{prop}"
    try:
        df_read=pd.read_csv(f'{dir}/{desc}/{model_name}/{prop}_{fold}.csv')
        # Combine
        df_combine=pd.concat([df_combine,df_read],
                            axis=1)
    except:
        pass
        
    # count=0
stats_list=['r2','rmsd','bias','SDEP','gradient','b','mse','mae']
for stat in stats_list:
    columns_to_compute = df_combine.filter(regex=f'^{stat}_{prop}')
    # print(columns_to_compute)
    # Compute mean and std for the specified columns
    mean_values = df_combine.filter(regex=f'^{stat}_{prop}').mean(axis=1)
    std_values = df_combine.filter(regex=f'^{stat}_{prop}').std(axis=1)
    df_combine_all[f'{stat}_{prop}_mean'] = mean_values
    df_combine_all[f'{stat}_{prop}_std']  = std_values
    if stat == 'r2' or stat =='rmsd':
        df_summary[f'{stat}_{prop}_mean'] = mean_values
        df_summary[f'{stat}_{prop}_std']  = std_values

df_combine = df_combine.loc[:,~df_combine.columns.duplicated()].copy()
df_summary['trial_id']=df_combine['Unnamed: 0']
#  Find the trial number with the best hyper paramteres
trial_number_fin_model = df_summary.loc[df_summary[f'r2_{prop}_mean'].idxmax()]['trial_id']

# Load best hyper parameters into a dataframe
best_hps=df_combine.loc[df_combine['Unnamed: 0']==trial_number_fin_model][df_combine.columns[2:7]]

# Load Hyper parameters for final model
hp = kt.HyperParameters()

model_type1 = hp.Choice("model_type1", [best_hps.values[0, 0]])
model_type = hp.Choice("model_type", [best_hps.values[0, 1]])
hp_layer_1= hp.Choice(f'layer_1', values=[best_hps.values[0, 4]])
hp_layer_2_2= hp.Choice(f'layer_2_2', values=[best_hps.values[0, 3]])
hp_layer_3_3= hp.Choice(f'layer_3_3',  values=[best_hps.values[0, 2]])

# Build final model
test_model=build_model(hp)

# Inputs for training final Model
epochs = 300
# batch  = 16
# TODO: Use path lib to create path
resample_path = path_resample(home,resample)

# model_name = architecture of - Single task/RF/KNN + dH/dG/dS/Tm or - Multitask  
# prop = 'dH' 
# model_name = f"1DConv_st_{prop}" 


# Define the directory path
directory_path = Path(f"{os.getcwd()}/CV/{resample}/{desc}/{model_name}/")

# Define Paths for call backs
csv_logger_path  = Path(f'{directory_path}/csv_logger/')
cp_callback_path = Path(f'{directory_path}/model_checkpoint/')
tensorboard_path = Path(f'{directory_path}/tensorboard_logs/')

# Ensure the directory exists, create it if necessary
csv_logger_path.mkdir(parents=True, exist_ok=True)
cp_callback_path.mkdir(parents=True, exist_ok=True)
tensorboard_path.mkdir(parents=True, exist_ok=True)


#### CALL BACKS!
es = EarlyStopping(monitor      = 'val_loss', 
                        mode     = 'min', 
                        verbose  = 1, 
                        patience = 2000, 
                    restore_best_weights = True)
# CSV Logger
csv_logger = CSVLogger(f'{csv_logger_path}/model_history.csv' , append=True)

# CP_callbacks      not required when using a tunner       
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'{cp_callback_path}/cp.ckpt',
#                                                  save_weights_only=True,
#                                                  verbose=1)

# TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_path, 
                                                       #/{batch}', # _ADAPTIVELEARNIGNRATE_01_10_Dense3_64_3CNN_lr_3_es
                                                      update_freq = 1,
                                                      # histogram_freq=1, 
                                                      write_graph=False, 
                                                      write_images=False)
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard

# Covert to list and provide to Keras Regressor
keras_callbacks = [es, csv_logger, tensorboard_callback]

# Load resample data
y_1_train, y_2_train, y_3_train, y_4_train, Y_train, X_padded_train, X_train = load_xy(train,desc)
y_1_val,   y_2_val,   y_3_val,   y_4_val,   Y_val,   X_padded_val,   X_val   = load_xy(val,desc)
y_1_test,  y_2_test,  y_3_test,  y_4_test,  Y_test,  X_padded_test,  X_test  = load_xy(test,desc)

# Train model on resample data

ys_train = [y_1_train,y_2_train,y_3_train,y_4_train] 
ys_val   = [y_1_val,y_2_val,y_3_val,y_4_val]
ys_test  = [y_1_test,y_2_test,y_3_test,y_4_test]
GSHT_list.index(f'{prop}')+1

history=test_model.fit(X_padded_train, ys_train[GSHT_list.index(f'{prop}')],
            epochs = epochs,
            batch_size=batch,
            verbose = 2,
            validation_data =(X_padded_val, ys_val[GSHT_list.index(f'{prop}')]),
             # validation_split = 0.2,
            callbacks=keras_callbacks)
# Make predictions


# Store stats


# Store csv files (predictions vs true)


def stats_csv(y_test_pred, Y_test, prop, csv_name, set_type,resample):
    y_test_np = Y_test[f'{prop}'].to_numpy()
    y_pred_test_np = y_test_pred

    r2_test = r2_score(y_test_np, y_pred_test_np)
    rmsd_test = (mean_squared_error(y_test_np, y_pred_test_np))**0.5
    bias_test = np.mean(y_pred_test_np - y_test_np)
    sdep_test = (np.mean((y_pred_test_np - y_test_np - bias_test)**2))**0.5
    r2 = '{:.3f}'.format(r2_test)
    rmsd = '{:.3f}'.format(rmsd_test)
    bias = '{:.3f}'.format(bias_test)
    sdep = '{:.3f}'.format(sdep_test)
    
    
    mse=mean_squared_error(y_test_np, y_pred_test_np)
    mse='{:.3f}'.format(mse)
    mae=mean_absolute_error(y_test_np, y_pred_test_np)
    mae='{:.3f}'.format(mae)
    try:
        a, b = np.polyfit(y_test_np, y_pred_test_np, 1)
        plot_a = '{:.3f}'.format(a[0])
        plot_b = '{:.3f}'.format(b[0])
    except np.linalg.LinAlgError:
        pass
        # 'index':np.array(Y_test.index),'ID':np.array(Y_test.index +1),
    y_pred_test_np = y_test_pred.squeeze()
    csv_df=pd.DataFrame({'index':np.array(Y_test.index),'ID':np.array(Y_test.index +1),'y_true': y_test_np, 'y_pred': y_pred_test_np})
    csv_df.to_csv(csv_name,index=None)

    csv_df_stats=pd.DataFrame(data=np.array([[resample, r2, rmsd, bias, sdep, plot_a, plot_b, mse, mae]]),
                              # index=None, 
                              columns=['resample','r2','rmsd', 'bias', 'sdep', 'plot_a', 'plot_b', 'mse', 'mae'])
    csv_df_stats.to_csv(f'{csv_name[:-4]}_stats.csv',mode='w',index='resample')
    
    return csv_df



set_types = ['train','val','test']
# set_type = 'test'
# csv_name=f'{directory_path}/{set_type}.csv'
# csv_name=f'{set_type}.csv'

list_indexes = [train, val, test]
index_counter=-1
for indexes in list_indexes:
    index_counter+=1
    csv_name=f'{directory_path}/{set_types[index_counter]}.csv'
    y_1,  y_2,  y_3,  y_4,  Y,  X_padded,  X  = load_xy(indexes,desc)
    predictions = test_model.predict(X_padded)
    csv_df = stats_csv(predictions, Y, prop, csv_name, set_types[index_counter],resample)



""")
        f.close
        


# Bash Script

In [193]:
def bash_script(resample, fold, desc, prop):
    cores=1
    HH=1
    MM=30
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st"
    python_script=f"{job_name}.py"
    bash_script=f"sbatch_{job_name}.sh"
    with open(f'{dir}/{bash_script}','w') as f:
        f.write(f"""#!/bin/bash

#======================================================
#
# Job script for running on a single node
#
#======================================================

#======================================================
# Propogate environment variables to the compute node
#SBATCH --export=ALL
#
# Run in the standard partition (queue)
#SBATCH --partition=standard
#
# Specify project account
#SBATCH --account=palmer-addmd
#
# No. of tasks required
#SBATCH --ntasks=1 --cpus-per-task={cores}
#
# Distribute processes in round-robin fashion for load balancing
#SBATCH --distribution=cyclic
#
#
# Specify (hard) runtime (HH:MM:SS)
#SBATCH --time=0{HH}:00:00
#
# Job name
#SBATCH --job-name={job_name}
#
# Output file
#SBATCH --output={job_name}_slurm-%j.out
#======================================================

module purge
module load anaconda/python-3.9.7/2021.11
source activate tf
# module purge

#======================================================
# Prologue script to record job details
# Do not change the line below
#======================================================
/opt/software/scripts/job_prologue.sh
#------------------------------------------------------

python {python_script}
mv {python_script} scripts
mv {bash_script} scripts
mv {job_name}_slurm-*.out scripts


#======================================================
# Epilogue script to record job endtime and runtime
# Do not change the line below
#======================================================
/opt/software/scripts/job_epilogue.sh
#------------------------------------------------------
""")
        f.close
        

### Execute Arch and Delete

In [198]:
def bash_script_arch_del(resample, fold, desc, prop):
    cores=1
    MM=59
    HH=2
    
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st_arch"
    python_script=f"{job_name}.py"
    bash_script=f"sbatch_{job_name}.sh"
    with open(f'{dir}/{bash_script}','w') as f:
        f.write(f"""#!/bin/bash

#======================================================
#
# Job script for running on a single node
#
#======================================================

#======================================================
# Propogate environment variables to the compute node
#SBATCH --export=ALL
#
# Run in the standard partition (queue)
#SBATCH --partition=standard
#
# Specify project account
#SBATCH --account=palmer-addmd
#
# No. of tasks required
#SBATCH --ntasks=1 --cpus-per-task={cores}
#
# Distribute processes in round-robin fashion for load balancing
#SBATCH --distribution=cyclic
#
#
# Specify (hard) runtime (HH:MM:SS)
#SBATCH --time=0{HH}:{MM}:00
#
# Job name
#SBATCH --job-name={job_name}
#
# Output file
#SBATCH --output={job_name}_slurm-%j.out
#======================================================

module purge
module load anaconda/python-3.9.7/2021.11
source activate tf
module purge

#======================================================
# Prologue script to record job details
# Do not change the line below
#======================================================
/opt/software/scripts/job_prologue.sh
#------------------------------------------------------

python {python_script}
mv {python_script} scripts
mv {bash_script} scripts
mv {job_name}_slurm-*.out scripts


#======================================================
# Epilogue script to record job endtime and runtime
# Do not change the line below
#======================================================
/opt/software/scripts/job_epilogue.sh
#------------------------------------------------------
""")
        f.close
        

## Execute  Tempalte 3

In [149]:
def bash_script_temp4(resample, desc, prop):
    cores=1
    MM=30
    
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_st"
    python_script=f"{job_name}.py"
    bash_script=f"sbatch_{job_name}.sh"
    with open(f'{dir}/{bash_script}','w') as f:
        f.write(f"""#!/bin/bash

#======================================================
#
# Job script for running on a single node
#
#======================================================

#======================================================
# Propogate environment variables to the compute node
#SBATCH --export=ALL
#
# Run in the standard partition (queue)
#SBATCH --partition=standard
#
# Specify project account
#SBATCH --account=palmer-addmd
#
# No. of tasks required
#SBATCH --ntasks=1 --cpus-per-task={cores}
#
# Distribute processes in round-robin fashion for load balancing
#SBATCH --distribution=cyclic
#
#
# Specify (hard) runtime (HH:MM:SS)
#SBATCH --time=00:{MM}:00
#
# Job name
#SBATCH --job-name={job_name}
#
# Output file
#SBATCH --output={job_name}_slurm-%j.out
#======================================================

module purge
module load anaconda/python-3.9.7/2021.11
source activate tf
module purge

#======================================================
# Prologue script to record job details
# Do not change the line below
#======================================================
/opt/software/scripts/job_prologue.sh
#------------------------------------------------------

python {python_script}
mv {python_script} scripts
mv {bash_script} scripts
mv {job_name}_slurm-*.out scripts


#======================================================
# Epilogue script to record job endtime and runtime
# Do not change the line below
#======================================================
/opt/software/scripts/job_epilogue.sh
#------------------------------------------------------
""")
        f.close
        

## Execute Template 4

In [ ]:
def bash_script(resample, fold, desc, prop):
    cores=1
    HH=24
    
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st"
    python_script=f"{job_name}.py"
    bash_script=f"sbatch_{job_name}.sh"
    with open(f'{dir}/{bash_script}','w') as f:
        f.write(f"""#!/bin/bash

#======================================================
#
# Job script for running on a single node
#
#======================================================

#======================================================
# Propogate environment variables to the compute node
#SBATCH --export=ALL
#
# Run in the standard partition (queue)
#SBATCH --partition=standard
#
# Specify project account
#SBATCH --account=palmer-addmd
#
# No. of tasks required
#SBATCH --ntasks=1 --cpus-per-task={cores}
#
# Distribute processes in round-robin fashion for load balancing
#SBATCH --distribution=cyclic
#
#
# Specify (hard) runtime (HH:MM:SS)
#SBATCH --time={HH}:00:00
#
# Job name
#SBATCH --job-name={job_name}
#
# Output file
#SBATCH --output={job_name}_slurm-%j.out
#======================================================

module purge
module load anaconda/python-3.9.7/2021.11
source activate tf
# module purge

#======================================================
# Prologue script to record job details
# Do not change the line below
#======================================================
/opt/software/scripts/job_prologue.sh
#------------------------------------------------------

python {python_script}
mv {python_script} scripts
mv {bash_script} scripts
mv {job_name}_slurm-*.out scripts


#======================================================
# Epilogue script to record job endtime and runtime
# Do not change the line below
#======================================================
/opt/software/scripts/job_epilogue.sh
#------------------------------------------------------
""")
        f.close
        

# Job Submission

In [5]:
import os

In [6]:
abs_dir='/users/qdb16186/CNN_stk'

desc_type = ['DNA-Groups','Granulated','OHEP','LP_dec2']
GSHT_list=['dH','dS','dG','Tm']
# GSHT='dH_dS_dG_Tm' 
desc='Granulated'
prop = 'dH'
os.chdir('/users/qdb16186/CNN_stk')

resample=0
fold=0
python_script(resample, fold, desc, prop)
bash_script(resample, fold, desc, prop)

job_name=f"{desc}_{prop}_{resample}_{fold}_st"

# job_name=f"{job_resample}_{job_i_fold}_{prop}_CNN_multi_task_{GSHT}_{cores}"
print(f'{abs_dir}/sbatch_{job_name}.sh')
bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
os.system(bashpermissions)
bashCommand = f"sbatch sbatch_{job_name}.sh"
os.system(bashCommand)


/users/qdb16186/CNN_stk/sbatch_Granulated_dH_0_0_st.sh
Submitted batch job 10797643


0

In [7]:
GSHT_list[1:4]

['dS', 'dG', 'Tm']

In [8]:
model_name=f'1DConv_st_{prop}'

f'{os.getcwd()}/CV/{resample}/{fold}/{desc}/{model_name}'



'/users/qdb16186/CNN_stk/CV/0/0/Granulated/1DConv_st_dH'

## Delete wrongly submited jobs for hyper paramtere tuning
error was that the training and validation set were set to y_1_(test/val) which measn it was only dH not appropriately set to dS dG and Tm afterwwards.
the fix was to to 1) define GSHT_List, 2) index the GSHT_list based on the prop + 1 to get the correct list index for 3) combine y_1-4_test/val/train in ys_test/val/train =[] list and index it using 2). in the history

In [20]:
def bash_delete(resample, fold, desc, prop):
    cores=1
    HH=30
    model_name = f'1DConv_st_{prop}'
    dir=os.getcwd()
    job_name=f"{desc}_{prop}_{resample}_{fold}_st"
    python_script=f"{job_name}.py"
    bash_script=f"sbatch_{job_name}.sh"
    with open(f'{dir}/{bash_script}','w') as f:
        f.write(f"""#!/bin/bash

#======================================================
#
# Job script for running on a single node
#
#======================================================

#======================================================
# Propogate environment variables to the compute node
#SBATCH --export=ALL
#
# Run in the standard partition (queue)
#SBATCH --partition=standard
#
# Specify project account
#SBATCH --account=palmer-addmd
#
# No. of tasks required
#SBATCH --ntasks=1 --cpus-per-task={cores}
#
# Distribute processes in round-robin fashion for load balancing
#SBATCH --distribution=cyclic
#
#
# Specify (hard) runtime (HH:MM:SS)
#SBATCH --time=00:{HH}:00
#
# Job name
#SBATCH --job-name={job_name}
#
# Output file
#SBATCH --output={job_name}_slurm-%j.out
#======================================================
""")
        f.close
        f.write(
f'rm -r {os.getcwd()}/CV/{resample}/{fold}/{desc}/{model_name}')
        f.close
        f.write(f"""
#======================================================
# Prologue script to record job details
# Do not change the line below
#======================================================
/opt/software/scripts/job_prologue.sh
#------------------------------------------------------

mv {python_script} scripts
mv {bash_script} scripts
mv {job_name}_slurm-*.out scripts


#======================================================
# Epilogue script to record job endtime and runtime
# Do not change the line below
#======================================================
/opt/software/scripts/job_epilogue.sh
#------------------------------------------------------
""")
        f.close
        

In [ ]:
# Cell for deleting faulty runs/jobs

# desc='Granulated'
# os.chdir('/users/qdb16186/CNN_stk')
# for prop in GSHT_list[1:4]:
#     for resample in range(0,10):
#         for fold in range(5):
#             bash_delete(resample, fold, desc, prop)
#             job_name=f"{desc}_{prop}_{resample}_{fold}_st"
#             bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
#             os.system(bashpermissions)
#             bashCommand = f"sbatch sbatch_{job_name}.sh"
#             os.system(bashCommand)
            # os.system(f'rm -r {os.getcwd()}/CV/{resample}/{fold}/{desc}/{model_name}')

## Submitting Hyper paramtere Tuning

In [184]:
desc='H-Bonding'
os.chdir('/users/qdb16186/CNN_stk')
for prop in GSHT_list:
    for resample in range(0,50):
        for fold in range(5):
            
            python_script(resample, fold, desc, prop)
            bash_script(resample, fold, desc, prop)
            job_name=f"{desc}_{prop}_{resample}_{fold}_st"
            bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
            os.system(bashpermissions)
            bashCommand = f"sbatch sbatch_{job_name}.sh"
            os.system(bashCommand)

Submitted batch job 10854459
Submitted batch job 10854460
Submitted batch job 10854461
Submitted batch job 10854462
Submitted batch job 10854463
Submitted batch job 10854464
Submitted batch job 10854465
Submitted batch job 10854466
Submitted batch job 10854467
Submitted batch job 10854468
Submitted batch job 10854469
Submitted batch job 10854470
Submitted batch job 10854471
Submitted batch job 10854472
Submitted batch job 10854473
Submitted batch job 10854474
Submitted batch job 10854475
Submitted batch job 10854476
Submitted batch job 10854477
Submitted batch job 10854478
Submitted batch job 10854479
Submitted batch job 10854480
Submitted batch job 10854481
Submitted batch job 10854482
Submitted batch job 10854483
Submitted batch job 10854484
Submitted batch job 10854485
Submitted batch job 10854486
Submitted batch job 10854487
Submitted batch job 10854488
Submitted batch job 10854489
Submitted batch job 10854490
Submitted batch job 10854491
Submitted batch job 10854492
Submitted batc

In [16]:
bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
os.system(bashpermissions)

chmod: cannot access '/users/qdb16186/CNN_stk/sbatch_Granulated_dH_0_0_st.sh': No such file or directory


256

In [108]:
for i in range(10803719,10803768):
    
    cancel = f"scancel {i}"
    os.system(cancel)

## Sbumitting Grid Search Analysis

In [134]:
GSHT_list

['dH', 'dS', 'dG', 'Tm']

In [187]:
desc='CountDNAp'
os.chdir('/users/qdb16186/CNN_stk')
for prop in GSHT_list:
    for resample in range(0,50):
        for fold in range(5):
            python_grid_analysis(resample, fold, desc, prop)
            bash_script(resample, fold, desc, prop)
            job_name=f"{desc}_{prop}_{resample}_{fold}_st"
            bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
            os.system(bashpermissions)
            bashCommand = f"sbatch sbatch_{job_name}.sh"
            os.system(bashCommand)

Submitted batch job 10857962
Submitted batch job 10857963
Submitted batch job 10857964
Submitted batch job 10857965
Submitted batch job 10857966
Submitted batch job 10857967
Submitted batch job 10857968
Submitted batch job 10857969
Submitted batch job 10857970
Submitted batch job 10857971
Submitted batch job 10857972
Submitted batch job 10857973
Submitted batch job 10857974
Submitted batch job 10857975
Submitted batch job 10857976
Submitted batch job 10857977
Submitted batch job 10857978
Submitted batch job 10857979
Submitted batch job 10857980
Submitted batch job 10857981
Submitted batch job 10857982
Submitted batch job 10857983
Submitted batch job 10857984
Submitted batch job 10857985
Submitted batch job 10857986
Submitted batch job 10857987
Submitted batch job 10857988
Submitted batch job 10857989
Submitted batch job 10857990
Submitted batch job 10857991
Submitted batch job 10857992
Submitted batch job 10857993
Submitted batch job 10857994
Submitted batch job 10857995
Submitted batc

In [235]:
7*5*50*4

7000

## Archiving and Deleting Folders

In [239]:
7*5*50*4


# os.chdir('/users/qdb16186/CNN_stk')
# for desc in desc_type:
#     for prop in GSHT_list:
#         # print(prop)
#         for resample in range(0,50):
#             for fold in range(5):
count=0
progress =7000
desc_type = ['Granulated', 'OHEP', 'LP_dec2', 'DNA-Groups','RF-Score', 'CountDNAp', 'H-Bonding']
resubmit_jobs=[]
for desc in desc_type:
    for prop in GSHT_list:
        for resample in range(50):
            for fold in range(5):
                # for root, dirs, files in os.walk(f'/users/qdb16186/CNN_stk/CV/{resample}/{fold}/{desc}//1DConv_st_{prop}'):
                    entries = os.listdir(f'/users/qdb16186/CNN_stk/CV/{resample}/{fold}/{desc}//1DConv_st_{prop}/')
                    # print(root,dirs,files)
                    # print(entries)
                    progress-=1
                    print(progress)
                    if 'tuner_results.tar.gz' not in entries:
                        resubmit_jobs.append([resample,fold,desc,prop])
                        count+=1
count

6999
6998
6997
6996
6995
6994
6993
6992
6991
6990
6989
6988
6987
6986
6985
6984
6983
6982
6981
6980
6979
6978
6977
6976
6975
6974
6973
6972
6971
6970
6969
6968
6967
6966
6965
6964
6963
6962
6961
6960
6959
6958
6957
6956
6955
6954
6953
6952
6951
6950
6949
6948
6947
6946
6945
6944
6943
6942
6941
6940
6939
6938
6937
6936
6935
6934
6933
6932
6931
6930
6929
6928
6927
6926
6925
6924
6923
6922
6921
6920
6919
6918
6917
6916
6915
6914
6913
6912
6911
6910
6909
6908
6907
6906
6905
6904
6903
6902
6901
6900
6899
6898
6897
6896
6895
6894
6893
6892
6891
6890
6889
6888
6887
6886
6885
6884
6883
6882
6881
6880
6879
6878
6877
6876
6875
6874
6873
6872
6871
6870
6869
6868
6867
6866
6865
6864
6863
6862
6861
6860
6859
6858
6857
6856
6855
6854
6853
6852
6851
6850
6849
6848
6847
6846
6845
6844
6843
6842
6841
6840
6839
6838
6837
6836
6835
6834
6833
6832
6831
6830
6829
6828
6827
6826
6825
6824
6823
6822
6821
6820
6819
6818
6817
6816
6815
6814
6813
6812
6811
6810
6809
6808
6807
6806
6805
6804
6803
6802
6801
6800


2635

In [244]:
len(resubmit_jobs)
resubmit_jobs[0]

[35, 4, 'RF-Score', 'dH']

In [245]:
for resub_info in resubmit_jobs:
    print(resub_info)
    resample=resub_info[0]
    fold=resub_info[1]
    desc=resub_info[2]
    prop=resub_info[3]
    python_archive_del_hp(resample, fold, desc, prop)
    bash_script_arch_del(resample, fold, desc, prop)
    job_name=f"{desc}_{prop}_{resample}_{fold}_st_arch"
    bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
    os.system(bashpermissions)
    bashCommand = f"sbatch sbatch_{job_name}.sh"
    os.system(bashCommand)

[35, 4, 'RF-Score', 'dH']
Submitted batch job 10879072
[36, 0, 'RF-Score', 'dH']
Submitted batch job 10879073
[36, 1, 'RF-Score', 'dH']
Submitted batch job 10879074
[36, 2, 'RF-Score', 'dH']
Submitted batch job 10879075
[36, 3, 'RF-Score', 'dH']
Submitted batch job 10879076
[36, 4, 'RF-Score', 'dH']
Submitted batch job 10879077
[37, 0, 'RF-Score', 'dH']
Submitted batch job 10879078
[37, 1, 'RF-Score', 'dH']
Submitted batch job 10879079
[37, 2, 'RF-Score', 'dH']
Submitted batch job 10879080
[37, 3, 'RF-Score', 'dH']
Submitted batch job 10879081
[37, 4, 'RF-Score', 'dH']
Submitted batch job 10879082
[38, 0, 'RF-Score', 'dH']
Submitted batch job 10879083
[38, 1, 'RF-Score', 'dH']
Submitted batch job 10879084
[38, 2, 'RF-Score', 'dH']
Submitted batch job 10879085
[38, 3, 'RF-Score', 'dH']
Submitted batch job 10879086
[38, 4, 'RF-Score', 'dH']
Submitted batch job 10879087
[39, 0, 'RF-Score', 'dH']
Submitted batch job 10879088
[39, 1, 'RF-Score', 'dH']
Submitted batch job 10879089
[39, 2, 'R

In [201]:
desc='OHEP'
desc_type = ['Granulated', 'OHEP', 'LP_dec2', 'DNA-Groups']
desc_type = ['RF-Score', 'CountDNAp', 'H-Bonding']
desc_type = ['Granulated', 'OHEP', 'LP_dec2', 'DNA-Groups','RF-Score', 'CountDNAp', 'H-Bonding']
os.chdir('/users/qdb16186/CNN_stk')
for desc in desc_type:
    for prop in GSHT_list:
        # print(prop)
        for resample in range(0,50):
            for fold in range(5):
                python_archive_del_hp(resample, fold, desc, prop)
                bash_script_arch_del(resample, fold, desc, prop)
                job_name=f"{desc}_{prop}_{resample}_{fold}_st_arch"
                bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
                os.system(bashpermissions)
                bashCommand = f"sbatch sbatch_{job_name}.sh"
                os.system(bashCommand)

# python_archive_del_hp(resample, fold, desc, prop):

Submitted batch job 10874806
Submitted batch job 10874807
Submitted batch job 10874808
Submitted batch job 10874809
Submitted batch job 10874810
Submitted batch job 10874811
Submitted batch job 10874812
Submitted batch job 10874813
Submitted batch job 10874814
Submitted batch job 10874815
Submitted batch job 10874816
Submitted batch job 10874817
Submitted batch job 10874818
Submitted batch job 10874819
Submitted batch job 10874820
Submitted batch job 10874821
Submitted batch job 10874822
Submitted batch job 10874823
Submitted batch job 10874824
Submitted batch job 10874825
Submitted batch job 10874826
Submitted batch job 10874827
Submitted batch job 10874828
Submitted batch job 10874829
Submitted batch job 10874830
Submitted batch job 10874831
Submitted batch job 10874832
Submitted batch job 10874833
Submitted batch job 10874834
Submitted batch job 10874835
Submitted batch job 10874836
Submitted batch job 10874837
Submitted batch job 10874838
Submitted batch job 10874839
Submitted batc

sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit l

Submitted batch job 10878900
Submitted batch job 10878901
Submitted batch job 10878902
Submitted batch job 10878903
Submitted batch job 10878904
Submitted batch job 10878905
Submitted batch job 10878906
Submitted batch job 10878907
Submitted batch job 10878908
Submitted batch job 10878909
Submitted batch job 10878910


sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)


Submitted batch job 10878911
Submitted batch job 10878912
Submitted batch job 10878913
Submitted batch job 10878914


...

sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit l

Submitted batch job 10879029


sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit l

Submitted batch job 10879030


sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit limit, user's size and/or time limits)
sbatch: error: QOSMaxSubmitJobPerUserLimit
sbatch: error: Batch job submission failed: Job violates accounting/QOS policy (job submit l

## Combine Grid Searhc and Archiving and Deleting

In [77]:
desc='Granulated'
os.chdir('/users/qdb16186/CNN_stk')
for prop in GSHT_list[0:1]:
    # print(prop)
    for resample in range(0,10):
        for fold in range(5):
            python_temp_2_3(resample, fold, desc, prop)
            bash_script(resample, fold, desc, prop)
            job_name=f"{desc}_{prop}_{resample}_{fold}_st"
            bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
            os.system(bashpermissions)
            bashCommand = f"sbatch sbatch_{job_name}.sh"
            os.system(bashCommand)
        

Submitted batch job 10800414
Submitted batch job 10800415
Submitted batch job 10800416
Submitted batch job 10800417
Submitted batch job 10800418
Submitted batch job 10800419
Submitted batch job 10800420
Submitted batch job 10800421
Submitted batch job 10800422
Submitted batch job 10800423
Submitted batch job 10800424
Submitted batch job 10800425
Submitted batch job 10800426
Submitted batch job 10800427
Submitted batch job 10800428
Submitted batch job 10800429
Submitted batch job 10800430
Submitted batch job 10800431
Submitted batch job 10800432
Submitted batch job 10800433
Submitted batch job 10800434
Submitted batch job 10800435
Submitted batch job 10800436
Submitted batch job 10800437
Submitted batch job 10800438
Submitted batch job 10800439
Submitted batch job 10800440
Submitted batch job 10800441
Submitted batch job 10800442
Submitted batch job 10800443
Submitted batch job 10800444
Submitted batch job 10800445
Submitted batch job 10800446
Submitted batch job 10800447
Submitted batc

In [110]:
GSHT_list[1:2]

['dS']

## Submit job - Access Best hyper parameters across 5 fold Then train and test on Resample data

In [190]:
desc='CountDNAp'
os.chdir('/users/qdb16186/CNN_stk')
for prop in GSHT_list:
    # print(prop)
    for resample in range(0,50):
        # for fold in range(5):
            python_temp_4(resample, desc, prop)
            bash_script_temp4(resample, desc, prop)
            # job_name=f"{desc}_{prop}_{resample}_{fold}_st"
            job_name=f"{desc}_{prop}_{resample}_st"
            bashpermissions = f"chmod u+x {abs_dir}/sbatch_{job_name}.sh"
            os.system(bashpermissions)
            bashCommand = f"sbatch sbatch_{job_name}.sh"
            os.system(bashCommand)

Submitted batch job 10859404
Submitted batch job 10859405
Submitted batch job 10859406
Submitted batch job 10859407
Submitted batch job 10859408
Submitted batch job 10859409
Submitted batch job 10859410
Submitted batch job 10859411
Submitted batch job 10859412
Submitted batch job 10859413
Submitted batch job 10859414
Submitted batch job 10859415
Submitted batch job 10859416
Submitted batch job 10859417
Submitted batch job 10859418
Submitted batch job 10859419
Submitted batch job 10859420
Submitted batch job 10859421
Submitted batch job 10859422
Submitted batch job 10859423
Submitted batch job 10859424
Submitted batch job 10859425
Submitted batch job 10859426
Submitted batch job 10859427
Submitted batch job 10859428
Submitted batch job 10859429
Submitted batch job 10859430
Submitted batch job 10859431
Submitted batch job 10859432
Submitted batch job 10859433
Submitted batch job 10859434
Submitted batch job 10859435
Submitted batch job 10859436
Submitted batch job 10859437
Submitted batc

## Untar file and delete or rename folders

In [89]:
desc='Granulated'
os.chdir('/users/qdb16186/CNN_stk')
for prop in GSHT_list[0:1]:
    # print(prop)
    for resample in range(0,10):
        for fold in range(5):
            os.chdir(f'/users/qdb16186/CNN_stk/CV/{resample}/{fold}/{desc}/1DConv_st_{prop}/')

            bashCommand = f"tar -xvzf tuner_results.tar.gz"
            os.system(bashCommand)
            # bashCommand = f"tar -xvzf tuner_results.tar.gz"
            # os.system(bashCommand)

In [104]:
GSHT_list

['dH', 'dS', 'dG', 'Tm']

In [127]:
desc='Granulated'
os.chdir('/users/qdb16186/CNN_stk')
# for prop in GSHT_list[0:1]:
    # print(prop)
prop='dH'
for resample in range(0,10):
    for fold in range(5):
        os.chdir(f'/users/qdb16186/CNN_stk/CV/{resample}/{fold}/{desc}/')
        # 1DConv_st_{prop}/')
        bashCommand = f"rm -r 1DConv_st_dH"
        os.system(bashCommand)
        # bashCommand = f"rm -r 1DConv_st_Tm"
        # os.system(bashCommand)
        # bashCommand = f"mv 1DConv_st_dG 1DConv_st_Tm"
        # os.system(bashCommand)
        # bashCommand = f"mv 1DConv_st_dS 1DConv_st_dG"
        # os.system(bashCommand)
        # bashCommand = f"mv 1DConv_st_dH 1DConv_st_dS"
        # os.system(bashCommand)

## Monitor Output files in a table


In [119]:
#  read scirt folder.
# Each file should be stored on a separate row
# columns should include: out file name, run time, comment section, last line and lst 5 or 10 lines of comment sections